In [205]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [206]:
import numpy as np
import pandas as pd
from pathlib import Path

In [207]:
import sys
sys.path.append("/dsmlp/home-fs04/19/019/riling/scalable_rl_portfolio_management")


In [208]:
from tics.tic_config import tics_176, tics_grouped
from agent.data_downloader import short_name_sha256

In [209]:
train_start_date= '2009-01-01'
train_end_date= '2020-07-01'
# test_start_date= '2020-07-01'
test_start_date = '2020-09-14'
test_end_date= '2024-10-01'

In [210]:
start = train_start_date
end = test_end_date

In [224]:
df_baseline = pd.read_csv(Path("acount_value") / "dow_jones_data.csv")[['close', 'date']].rename(columns={'close': 'account_value'})
df_manager = pd.read_csv(Path("acount_value") / "manager.csv")[['account_value', 'date']]
# df_176 = pd.read_csv(Path("acount_value") / "176.csv")[['account_value', 'date']]

In [225]:
dfs = []
tics = []
for i in range(6):
    tics += tics_grouped[i]
    name = short_name_sha256('_'.join(tics_grouped[i]))
    vaulue_data_file = Path("acount_value") / f"{name}_{start}_{end}.csv"
    value_df = pd.read_csv(vaulue_data_file)[['account_value', 'date']].drop_duplicates()
    dfs.append(value_df)

In [226]:
def normalize_from(df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    start = pd.to_datetime(start_date)
    end   = pd.to_datetime(end_date)

    # 1. 筛选并排序
    mask = (df['date'] >= start) & (df['date'] <= end)
    filtered = df.loc[mask].sort_values('date')

    # 2. 如果没有在 start_date 之后（含）及 end_date 之前的记录，报错
    if filtered.empty:
        raise ValueError(f"没有找到日期区间 {start_date} 到 {end_date} 的记录")

    # 3. 取第一个日期的 account_value 作为 v0
    v0 = filtered.iloc[0]['account_value']

    # 4. 归一化
    filtered['account_value'] = filtered['account_value'] / v0

    return filtered

In [327]:
import plotly.graph_objects as go
import plotly.express as px

def plot_account_value_comparison_plotly(
    models = None,
    model_labels = None,
    baseline = None,
    baseline_label = None,
    manager = None,
    manager_lable = None,
    x_col: str = 'date',
    y_col: str = 'account_value',
    title: str = None,
) -> go.Figure:
    """
    用 Plotly 绘制多条模型的归一化账户价值（收益率）曲线及一条基准曲线，
    其中所有 y_col 已从 1 起归一化，代表收益率曲线。

    参数
    ----
    models : list[pd.DataFrame]
        要对比的模型列表，每个 DataFrame 至少包含 x_col 和 y_col 两列。
    model_labels : list[str]
        与 models 一一对应的图例标签。
    baseline : pd.DataFrame
        基准模型的 DataFrame，结构同上。
    baseline_label : str
        基准模型的图例标签。
    x_col : str, optional
        用作横轴的列名，默认为 'date'。
    y_col : str, optional
        用作纵轴的列名，默认为 'account_value'。
    title : str, optional
        图表标题，默认为 "Normalized Return Comparison"。
    """


    fig = go.Figure()

    if models is not None:
        color_sequence = px.colors.qualitative.Pastel

        for i, (df, label) in enumerate(zip(models, model_labels)):
            # days = list(range(1, len(df) + 1))
            fig.add_trace(
                go.Scatter(
                    x=df[x_col],
                    # x=days,
                    y=df[y_col],
                    mode='lines',
                    name=label,
                    line=dict(color=color_sequence[i % len(color_sequence)], width=2),
                    opacity=0.6
                )
            )

    if manager is not None:
        # days = list(range(1, len(manager) + 1))
        fig.add_trace(
            go.Scatter(
                x=manager[x_col],
                # x=days,
                y=manager[y_col],
                mode='lines',
                name=manager_lable,
                line=dict(color='red')
            )
        )

    if baseline is not None:
        # days = list(range(1, len(baseline) + 1))
        fig.add_trace(
            go.Scatter(
                x=baseline[x_col],
                # x=days,
                y=baseline[y_col],
                mode='lines',
                name=baseline_label,
                line=dict(color='blue')
            )
        )

    scale = 2.4
    base_font = fig.layout.font.size or 12  # 默认12，如果你之前没设置过的话
    new_size = base_font * scale

    fig.update_layout(
        width=1200,
        height=800,
        # margin=dict(l=40, r=40, t=40, b=40),
        title=title,
        xaxis_title=x_col.capitalize(),
        yaxis_title="Total Return Rate",
        template="plotly_white",
        font=dict(
            size=new_size,         # 全局文字大小
            family="Time New Roman"  # 可选：指定字体系列
        ),
        title_font=dict(size=new_size * 1.2),    # 标题稍大一点
        legend_font=dict(size=new_size),         # 图例文字
        xaxis=dict(
            title_font=dict(size=new_size),
            tickfont=dict(size=new_size * 0.6)   # 刻度文字可以略小
        ),
        yaxis=dict(
            title_font=dict(size=new_size),
            tickfont=dict(size=new_size * 0.6)
        )
    )

    return fig

In [328]:
def plot(df_baseline, dfs, df_manager, test_start_date_s, test_end_date_s):

    test_start_date = pd.to_datetime(test_start_date_s)
    test_end_date = pd.to_datetime(test_end_date_s)

    df_baseline_normalized = normalize_from(df_baseline, test_start_date, test_end_date)
    dfs_normalized = [normalize_from(df, test_start_date, test_end_date) for df in dfs]
    df_manager_normalized = normalize_from(df_manager, test_start_date, test_end_date)

    fig1 = plot_account_value_comparison_plotly(
        models=dfs_normalized,
        model_labels=[f"Pool {i+1}" for i in range(len(dfs_normalized))],
        baseline=df_baseline_normalized,
        baseline_label="Dow Jones",
        title=f"Pool Return Comparison (since {test_start_date_s} to {test_end_date_s})"
        )
    fig2 = plot_account_value_comparison_plotly(
        baseline=df_baseline_normalized,
        baseline_label="Dow Jones",
        manager=df_manager_normalized,
        manager_lable="Manager",
        title=f"Manager Return Rate (since {test_start_date_s} to {test_end_date_s})"
    )

    return fig1, fig2

In [331]:
year = 2021
i = 1
test_start_date = f"{str(year)}-07-01"
test_end_date = f"{str(year+i)}-10-01"

In [332]:
fig1, fig2 = plot(df_baseline, dfs, df_manager, test_start_date, test_end_date)
fig1.show()
fig2.show()